# ML-Based Trade Filtering & Evaluation

This notebook applies a simple machine learning model to filter trade signals
based on engineered features and evaluates strategy performance.


## Objectives

The objectives of this notebook are:
- Load engineered feature data
- Define a simple supervised learning target
- Train a baseline ML classifier for trade filtering
- Evaluate model performance using appropriate metrics


## Modeling Philosophy

The goal is not to build a complex predictive model, but to demonstrate how
machine learning can be used as a probabilistic filter on top of rule-based
signals to improve decision quality and robustness.


In [1]:
import pandas as pd
import numpy as np
import os

from IPython.display import display

# Path to processed features
PROCESSED_DATA_PATH = os.path.join("..", "data", "processed")
features_path = os.path.join(PROCESSED_DATA_PATH, "features_1y.csv")

# Load feature dataset
df = pd.read_csv(features_path, parse_dates=["date"])

print("Feature dataset shape:", df.shape)
print("\nColumns:")
print(df.columns)

display(df.head())


Feature dataset shape: (17468, 9)

Columns:
Index(['date', 'close', 'ema_fast', 'ema_slow', 'ema_spread', 'trend_dir',
       'rolling_vol', 'high_vol', 'market_regime'],
      dtype='object')


,date,close,ema_fast,ema_slow,ema_spread,trend_dir,rolling_vol,high_vol,market_regime
0,2021-10-22 10:50:00+05:30,18233.65,18261.186100,18247.471314,13.714786,1,0.001021,1,bull_trend_high_vol
1,2021-10-22 10:55:00+05:30,18253.85,18260.487424,18247.721459,12.765965,1,0.000981,1,bull_trend_high_vol
2,2021-10-22 11:00:00+05:30,18271.70,18261.555288,18248.661794,12.893494,1,0.001004,1,bull_trend_high_vol
3,2021-10-22 11:05:00+05:30,18282.05,18263.507165,18249.971135,13.536030,1,0.000938,1,bull_trend_high_vol
4,2021-10-22 11:10:00+05:30,18262.35,18263.396959,18250.456581,12.940378,1,0.000944,1,bull_trend_high_vol


In [2]:
# in this case i amm going to predict if the close price will go up or down the next day
df["future_close"] = df["close"].shift(-1)
df["target"] = (df["future_close"] > df["close"]).astype(int)

# Drop last row (no future close available)
df = df.dropna().reset_index(drop=True)

print("Target variable created.")
print(df["target"].value_counts())

display(df[["date", "close", "future_close", "target"]].head())

Target variable created.
target
1    8949
0    8518
Name: count, dtype: int64


,date,close,future_close,target
0,2021-10-22 10:50:00+05:30,18233.65,18253.85,1
1,2021-10-22 10:55:00+05:30,18253.85,18271.70,1
2,2021-10-22 11:00:00+05:30,18271.70,18282.05,1
3,2021-10-22 11:05:00+05:30,18282.05,18262.35,0
4,2021-10-22 11:10:00+05:30,18262.35,18248.50,0


In [3]:
# Encode categorical features
df_encoded = pd.get_dummies(df, columns=["market_regime"], drop_first=True)

# Feature columns (exclude non-predictive fields)
feature_columns = [
    col for col in df_encoded.columns
    if col not in ["date", "close", "future_close", "target"]
]

X = df_encoded[feature_columns]
y = df_encoded["target"]

# Time-based train/test split (80% train, 20% test)
split_idx = int(len(df_encoded) * 0.8)

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print("Train size:", X_train.shape)
print("Test size :", X_test.shape)


Train size: (13973, 9)
Test size : (3494, 9)


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Build ML pipeline
model = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000))
])

# Train model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

print("Model training completed.")

Model training completed.


In [5]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", round(accuracy, 4))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Test Accuracy: 0.5074

Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.05      0.10      1728
           1       0.51      0.95      0.66      1766

    accuracy                           0.51      3494
   macro avg       0.51      0.50      0.38      3494
weighted avg       0.51      0.51      0.38      3494


Confusion Matrix:
[[  91 1637]
 [  84 1682]]


## Model Evaluation Summary

The baseline logistic regression model demonstrates modest predictive performance,
which is expected for high-frequency financial data. The results suggest that the
engineered features capture directional bias rather than precise price prediction.

This supports the use of machine learning as a probabilistic trade filter on top of
rule-based signals, rather than a standalone forecasting system.
